In [1]:
#%% General imports
import numpy as np
import torch

# DeepMoD stuff
from multitaskpinn import DeepMoD
from multitaskpinn.model.func_approx import Siren, NN
from multitaskpinn.model.library import Library1D
from multitaskpinn.model.constraint import LeastSquares
from multitaskpinn.model.sparse_estimators import Threshold 
from experiment_code import train, train_mt, train_gradnorm, train_scaled, train_wscaled, train_bayes, train_LL
from torch.distributions.studentT import StudentT
from deepymod_torch.training.sparsity_scheduler import TrainTestPeriodic
from scipy.io import loadmat

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%config InlineBackend.figure_format = 'svg'

# Setting cuda
if torch.cuda.is_available():
    device ='cuda'
else:
    device = 'cpu'

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


%load_ext autoreload
%autoreload 2

In [34]:
# Prepping data
data = loadmat('kuramoto_sivishinky.mat')

t = data['tt']
x = data['x']
u = data['uu']

# Normalizing data
t = (t - t.min())/(t.max()-t.min()) * 2 - 1
x = (x - x.min())/(x.max()-x.min()) * 2 - 1

x_grid, t_grid = np.meshgrid(x, t, indexing='ij')

# Limiting to non-chaotic part
lower_lim = 80
x_grid = x_grid[:, lower_lim:]
t_grid = t_grid[:, lower_lim:]
u = u[:, lower_lim:]

# %%Making training data
X = np.concatenate((t_grid.reshape(-1, 1), x_grid.reshape(-1, 1)), axis=1)
y = u.reshape(-1, 1)

# Adding noise
noise_level = 0.05
y_noisy = y + noise_level * np.std(y, axis=0) * np.random.randn(*y.shape)
number_of_samples = 25000

# Into tensor
idx = np.random.permutation(y.shape[0])
X_train = torch.tensor(X[idx, :][:number_of_samples], dtype=torch.float32).to(device)
y_train = torch.tensor(y_noisy[idx, :][:number_of_samples], dtype=torch.float32).to(device)

In [35]:
(noise_level * np.std(y, axis=0))**2

array([0.00359511])

In [36]:
# %%Configuring model
network = NN(2, [50, 50, 50, 50, 50, 50, 50, 50], 1) # Function approximator
library =  Library1D(poly_order=1, diff_order=4) # Library function
estimator = Threshold(0.1)#PDEFIND(lam=1e-6, dtol=0.1) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

# %% Setting schedulers
sparsity_scheduler = TrainTestPeriodic(patience=200, delta=1e-5)#Periodic(initial_epoch=10000, periodicity=100) # Defining when to apply sparsity
optimizer = torch.optim.Adam(model.parameters(), betas=(0.999, 0.999), lr=0.00025,  amsgrad=True) # Defining optimizer

In [7]:
train_mt(model, X_train, y_train, optimizer, sparsity_scheduler, log_dir='compare/reg_mse')

| Iteration | Progress | Time remaining |     Loss |      MSE |      Reg |    L1 norm |
       2050     20.50%             894s   6.96e-01   5.73e-01   3.50e-02   1.34e+01 

KeyboardInterrupt: 

In [39]:
# %%Configuring model
network = NN(2, [50, 50, 50, 50, 50, 50, 50, 50], 1) # Function approximator
library =  Library1D(poly_order=1, diff_order=4) # Library function
estimator = Threshold(0.1)#PDEFIND(lam=1e-6, dtol=0.1) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

# %% Setting schedulers
sparsity_scheduler = TrainTestPeriodic(patience=1000, delta=1e-5)#Periodic(initial_epoch=10000, periodicity=100) # Defining when to apply sparsity
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), lr=2e-3,  amsgrad=True) # Defining optimizer

In [40]:
train(model, X_train, y_train, optimizer, sparsity_scheduler, reg_weight=0.0, log_dir='compare/mse', max_iterations=2000)

| Iteration | Progress | Time remaining |     Loss |      MSE |      Reg |    L1 norm |
        300     15.00%             183s   1.35e+00   1.35e+00   2.28e+00   1.47e+00 

KeyboardInterrupt: 

In [9]:
from phimal_utilities.data import Dataset
from phimal_utilities.data.kdv import DoubleSoliton

In [10]:
# Making data
v = 0.1
A = 1.0
x = np.linspace(-5, 5, 100)
t = np.linspace(0.0, 1.0, 50)

x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(DoubleSoliton, c=[5.0, 2.0], x0=[-3.0, -1.0])
X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=2000, noise=0.2, normalize=True, random=True)
X, y = X.to(device), y.to(device)

In [13]:
# %%Configuring model
network = NN(2, [50, 50, 50, 50, 50, 50], 1) # Function approximator
library =  Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1)#PDEFIND(lam=1e-6, dtol=0.1) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

# %% Setting schedulers
sparsity_scheduler = TrainTestPeriodic(patience=8, delta=1e-5)#Periodic(initial_epoch=10000, periodicity=100) # Defining when to apply sparsity
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), lr=2e-3,  amsgrad=True) # Defining optimizer

In [14]:
train(model, X, y, optimizer, sparsity_scheduler, reg_weight=1e-5,log_dir='kdv/NN_b_1000')

| Iteration | Progress | Time remaining |     Loss |      MSE |      Reg |    L1 norm |
       1800     18.00%             317s   1.58e-02   1.58e-02   1.19e-01   5.69e+00 

In [15]:
model.estimator_coeffs()

[array([[ 0.        ],
        [-0.25648355],
        [ 0.        ],
        [-0.42956036],
        [ 0.        ],
        [-1.4069847 ],
        [ 0.        ],
        [ 0.25491226],
        [ 0.        ],
        [ 0.63230115],
        [ 0.        ],
        [ 0.        ]], dtype=float32)]

In [30]:
# %%Configuring model
network = Siren(2, [50, 50, 50, 50, 50, 50], 1) # Function approximator
library =  Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.0)#PDEFIND(lam=1e-6, dtol=0.1) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

# %% Setting schedulers
sparsity_scheduler = TrainTestPeriodic(patience=8, delta=1e-5)#Periodic(initial_epoch=10000, periodicity=100) # Defining when to apply sparsity
optimizer = torch.optim.Adam(model.parameters(), betas=(0.999, 0.999), lr=0.00025,  amsgrad=True) # Defining optimizer

In [31]:
train(model, X, y, optimizer, sparsity_scheduler, reg_weight=1,log_dir='kdv/NN_b_1000')

| Iteration | Progress | Time remaining |     Loss |      MSE |      Reg |    L1 norm |
       1550     15.50%             360s   1.92e-02   1.72e-02   2.00e-03   3.42e+00 

In [33]:
model.estimator_coeffs()

[array([[-0.        ],
        [-0.46877897],
        [-0.02032853],
        [-0.        ],
        [-0.1253418 ],
        [-0.6962671 ],
        [-0.        ],
        [ 0.18701857],
        [ 0.21026976],
        [ 0.13887863],
        [ 0.14900625],
        [-0.25000262]], dtype=float32)]